In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
from google.colab import drive
#drive.mount('/content/gdrive')

%matplotlib inline


import os
dataset='gdrive/My Drive/Colab Notebooks/dataset'

# Any results you write to the current directory are saved as output.

In [0]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
#train_labels, train_texts = get_labels_and_texts(dataset+'/train.ft.txt.bz2')

test_labels, test_texts = get_labels_and_texts(dataset+'/test.ft.txt.bz2')

In [16]:
j=0
for i in test_texts:
  j+=1
  print(i)
  print(" ")
  if j==6:
    break

Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"
 
One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs i

In [17]:
j=0
for i in test_labels:
  j+=1
  print(i)
  print(" ")
  if j==6:
    break

1
 
1
 
0
 
1
 
1
 
0
 


In [0]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        

test_texts = normalize_texts(test_texts)

In [0]:
MAX_FEATURES = 10000

#tokenizer = Tokenizer(num_words=MAX_FEATURES)
#tokenizer.fit_on_texts(train_texts)

import pickle
file= open('gdrive/My Drive/Colab Notebooks/tokenizer.pkl','rb')
tokenizer=pickle.load(file)
file.close()
test_texts = tokenizer.texts_to_sequences(test_texts)


In [0]:
MAX_LENGTH = 255
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)



In [0]:
#from keras import models 
root='gdrive/My Drive/'
#!fusermount -u drive
#!google-drive-ocamlfuse drive
import h5py
#rnn_model = models.load_model(dataset+'/../lstm.h5')
#lstm_model = load_model(root+'lstm.h5')
#bilstm_model = load_model(root+'bilstm.h5')

In [0]:
MAX_LENGTH=255
MAX_FEATURES=10000
def build_rnn_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.CuDNNGRU(128, return_sequences=True)(embedded)
    x = layers.CuDNNGRU(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
rnn_model = build_rnn_model()

In [24]:
rnn_model.load_weights('gdrive/My Drive/rnn.h5')
preds = rnn_model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))

InvalidArgumentError: ignored

In [0]:
def build_lstm_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.LSTM(128, return_sequences=True)(embedded)
    x = layers.LSTM(128)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
    
lstm_model = build_lstm_model()

In [26]:
lstm_model.load_weights('gdrive/My Drive/lstm.h5')
preds = lstm_model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))

InvalidArgumentError: ignored

In [0]:
def build_bilstm_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.LSTM(64, return_sequences=True)(embedded)
    x=layers.Dropout(0.2)(x)
    x = layers.Bidirectional(layers.CuDNNLSTM(128, return_sequences=True))(x)
    x=layers.Dropout(0.2)(x)
    x = layers.Bidirectional(layers.CuDNNLSTM(128, return_sequences=False))(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(100, activation='relu')(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    return model
bilstmmodel = build_bilstm_model()

In [28]:
bilstmmodel.load_weights('gdrive/My Drive/bilstm.h5')
preds = lstm_model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))

InvalidArgumentError: ignored

In [0]:
def classs(x):
  if x>0.5:
    return 1
  else:
    return 0
a=[]
for i in preds:
  a.append(classs(i[0]))

In [0]:
a=np.asarray(a)


In [0]:
tp=fp=fn=tn=0

for i in range(len(a)):
  #print(a[i])
  if test_labels[i]==1:
    if a[i]==1:
      tp+=1
    else:
      fn+=1
  else:
    if a[i]==0:
      fp+=1
    else:
      tn+=1

In [0]:
print(tp,fp,fn,tn,len(a))